In [1]:
import networkx as nx
import pandas as pd
import numpy as np

df_2018 = pd.read_csv('qajoin_2018.csv')

In [2]:
def com_detection(row_dataframe,limit_active):
    df = pd.DataFrame(row_dataframe, columns=['q_user','a_user'])
    df = df[-df['q_user'].isna()]
    dic_q = {}
    dic_a = {}
    qusers = [x for x in df['q_user'].to_numpy()]
    ausers = [x for x in df['a_user'].to_numpy()]
    for i in qusers:
        dic_q[i] = dic_q.get(i,0) + 1
    for i in ausers:
        dic_a[i] = dic_a.get(i,0) + 1
    dic_users = {}
    for i in list(dic_q.keys()) + list(dic_q.keys()):
        dic_users[i] = dic_q.get(i,0) + dic_a.get(i,0)
    dic_user_filtered = {k:v for k, v in dic_users.items() if v>= limit_active}
    list_user =  list(dic_user_filtered.keys())
    print("Number of Active User:",len(list_user))
    df_filter = df[(df['q_user'].isin(list_user)) & (df['a_user'].isin(list_user))]
    edges = [tuple(x) for x in df_filter.to_numpy()]
    dic = {}
    for i in edges:
        dic[i] = dic.get(i,0) + 1
    wei_edges = []
    for i in dic.keys():
        wei_edges.append((i[0],i[1],dic[i]))
    g = nx.Graph()
    g.add_weighted_edges_from(wei_edges)
    import community as community_louvain
    dendrogram = community_louvain.generate_dendrogram(g)
    partition = community_louvain.best_partition(g, weight = 'weight')
#    partition = community_louvain.partition_at_level(dendrogram, len(dendrogram) - 1)
    print("Number of Communities:",max(partition.values()))
    return(partition)

In [3]:
partition_2018 = com_detection(df_2018,20)
# partition_2018 = com_detection(df_2018,20)

Number of Active User: 26574
Number of Communities: 166


In [9]:
list_partition = list(partition_2018.values())
df_part = pd.DataFrame(list_partition)
df_count = df_part[0].value_counts()
df_count

2      4811
4      3587
7      2929
9      2165
5      2119
       ... 
89        1
105       1
121       1
137       1
103       1
Name: 0, Length: 167, dtype: int64

In [4]:
def community_tech(partition,top_num,row_dataframe):
    row_dataframe = row_dataframe.dropna(subset=['tags'])
    list_partition = list(partition.values())
    df_part = pd.DataFrame(list_partition)
    df_count = df_part[0].value_counts()
    top_commmunity = list(df_count.head(top_num).index)
#把community detection的结果变成 Community_id:[User_id]
    keys = pd.DataFrame(list(partition.keys()),columns=['User_id'])
    values = pd.DataFrame(list(partition.values()),columns=['Community_id'])
    df_community = pd.concat([keys,values],axis=1)
    dict_community = {}
    for i in list(df_community['Community_id'].unique()):
        dict_community[i] = list(df_community['User_id'].loc[df_community['Community_id'] == i])
    dict_community_tech = {}
    for i in top_commmunity:
        user_filter = dict_community[i]
        df_qa_community = row_dataframe[(row_dataframe['q_user'].isin(user_filter)) & (row_dataframe['a_user'].isin(user_filter))]
        dict_tech = {}
        dict_tech_orders = []
        for m in list(df_qa_community['tags']):
            techs = m.split('|')
            for j in techs:
                dict_tech[j] = dict_tech.get(j,0) + 1
        dict_tech_orders = sorted(dict_tech.items(), key=lambda x: x[1], reverse=True)
        dict_community_tech[i] = dict_tech_orders
        print('Number of questions in the community',i,len(df_qa_community))
    return(dict_community,dict_community_tech)

In [5]:
dict_community_result, dict_result = community_tech(partition_2018,10,df_2018)

Number of questions in the community 2 33210
Number of questions in the community 4 31394
Number of questions in the community 7 19400
Number of questions in the community 9 13406
Number of questions in the community 5 14013
Number of questions in the community 1 15056
Number of questions in the community 3 9762
Number of questions in the community 0 11092
Number of questions in the community 15 8150
Number of questions in the community 18 7882


In [6]:
import pickle
output1 = open('community_2018.pkl','wb')
output2 = open('tags_2018.pkl','wb')
pickle.dump(dict_community_result,output1)
pickle.dump(dict_result,output2)
output1.close()
output2.close()

In [7]:
for i in dict_result.keys():
    print('Community ',i)
    print(dict_result[i][:10])

Community  2
[('javascript', 12670), ('php', 9329), ('html', 6621), ('jquery', 5683), ('css', 5235), ('laravel', 5110), ('arrays', 2077), ('mysql', 1840), ('vue.js', 1436), ('laravel-5', 1310)]
Community  4
[('python', 24463), ('pandas', 9540), ('python-3.x', 5656), ('dataframe', 3148), ('django', 2748), ('numpy', 2278), ('list', 1648), ('dictionary', 1218), ('matplotlib', 845), ('tensorflow', 823)]
Community  7
[('c#', 8885), ('sql-server', 2713), ('sql', 2097), ('.net', 1206), ('asp.net-core', 1097), ('azure', 1084), ('asp.net', 1008), ('tsql', 901), ('asp.net-mvc', 857), ('xamarin.forms', 726)]
Community  9
[('java', 7755), ('spring', 2004), ('spring-boot', 1959), ('java-8', 867), ('java-stream', 569), ('hibernate', 510), ('maven', 484), ('spring-mvc', 404), ('spring-data-jpa', 393), ('jpa', 298)]
Community  5
[('android', 9425), ('java', 2293), ('flutter', 1594), ('kotlin', 1573), ('firebase', 1520), ('dart', 1040), ('android-studio', 706), ('android-recyclerview', 647), ('android-